## Import Libraries 

In [1]:
import pandas as pd
import requests
import zipfile
from io import BytesIO

## Set Display Options

In [2]:
pd.options.display.width = 0

## Download and Extract Data

In [4]:
zip_url = "https://fenixservices.fao.org/faostat/static/bulkdownloads/Production_Crops_Livestock_E_All_Data.zip"

# Name of the CSV file we want to extract from the zip file
csv_filename_without_extension = "Production_Crops_Livestock_E_All_Data_NOFLAG"

# HTTP GET request to download the zip file
response = requests.get(zip_url)
response.raise_for_status()

# Extract the specific CSV file from the zip file
with zipfile.ZipFile(BytesIO(response.content)) as zip_archive:
    # Find the filename in the archive
    for filename in zip_archive.namelist():
        if csv_filename_without_extension in filename:
            csv_filename = filename
            break
    else:
        raise ValueError(f"CSV file '{csv_filename_without_extension}' not found in the zip archive")

    with zip_archive.open(csv_filename) as csv_file:
        # Read the content of the CSV file into a DataFrame
        detected_encoding = "ISO-8859-1"  
        df = pd.read_csv(csv_file, encoding=detected_encoding)

display(df.head(5))

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Y1961,...,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021
0,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,ha,NaN,...,13490.0,14114.0,13703.0,14676.0,19481.0,19793.0,20053.0,29203.0,22134.0,21685.00
1,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5419,Yield,hg/ha,NaN,...,45960.0,29910.0,19996.0,16521.0,16859.0,13788.0,17161.0,13083.0,17759.0,18748.00
2,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5510,Production,tonnes,NaN,...,62000.0,42215.0,27400.0,24246.0,32843.0,27291.0,34413.0,38205.0,39307.0,40655.23
3,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5312,Area harvested,ha,NaN,...,18500.0,18500.0,30000.0,25000.0,24500.0,26160.0,25220.0,27387.0,26255.0,26287.00
4,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5419,Yield,hg/ha,NaN,...,6757.0,6757.0,7167.0,7200.0,7075.0,6970.0,7866.0,6902.0,7409.0,7379.00


## Exclude Years Before 2000

In [5]:
 #Getting the column names, so I can exclude years before 2000
column_names = df.columns

# List of years to be excluded
excluded_years = [f'Y{i}' for i in range(1961, 2000)]

# Drop excluded years
df = df.drop(columns=excluded_years)

display(df.head(5))

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Y2000,...,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021
0,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,ha,7000.0,...,13490.0,14114.0,13703.0,14676.0,19481.0,19793.0,20053.0,29203.0,22134.0,21685.00
1,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5419,Yield,hg/ha,17143.0,...,45960.0,29910.0,19996.0,16521.0,16859.0,13788.0,17161.0,13083.0,17759.0,18748.00
2,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5510,Production,tonnes,12000.0,...,62000.0,42215.0,27400.0,24246.0,32843.0,27291.0,34413.0,38205.0,39307.0,40655.23
3,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5312,Area harvested,ha,4000.0,...,18500.0,18500.0,30000.0,25000.0,24500.0,26160.0,25220.0,27387.0,26255.0,26287.00
4,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5419,Yield,hg/ha,6250.0,...,6757.0,6757.0,7167.0,7200.0,7075.0,6970.0,7866.0,6902.0,7409.0,7379.00


## Exclude Regions and Former Countries 

In [ ]:
# Get the unique values in the "Area" column to see what countries I need to exclude
countries_list = df['Area'].unique()

# Print the list of countries
for country in countries_list:
    print(country)
    
# Making the exclusion list for Area
regions_and_old_countries = ['Belgium-Luxembourg', 'China', 'Czechoslovakia','USSR', 'Yugoslav SFR', 'Serbia and Montenegro',
                             'World','Africa', 'Eastern Africa','Middle Africa', 'Northern Africa', 'Southern Africa',
                             'Western Africa', 'Americas', 'Northern America', 'Central America', 'Caribbean', 'South America',
                             'Asia','Central Asia', 'Eastern Asia', 'Southern Asia', 'South-eastern Asia', 'Western Asia',
                             'Europe', 'Eastern Europe', 'Northern Europe','Southern Europe', 'Western Europe','Oceania',
                             'Australia and New Zealand','Melanesia', 'Micronesia', 'Polynesia', 'European Union (27)', 
                             'Least Developed Countries', 'Land Locked Developing Countries','Small Island Developing States',
                             'Low Income Food Deficit Countries','Net Food Importing Developing Countries']
# Filtering conditions
conditions = (
    (df['Element'] == 'Production') &
    (df['Unit'] == 'tonnes') &
    (~df['Area'].isin(regions_and_old_countries)))

# Apply the conditions
df = df[conditions]

## Melt DataFrame 

In [6]:
# Melt the DataFrame from wide to long format
years = [f'Y{i}' for i in range(2000, 2022)]
df = pd.melt(df, id_vars=['Area Code', 'Area Code (M49)', 'Area', 'Item Code', 'Item',
                          'Element Code', 'Element', 'Unit'], value_vars=years)

display(df.head(5))

,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Unit,variable,value
0,2,'004,Afghanistan,221,"Almonds, in shell",5312,Area harvested,ha,Y2000,7000.0
1,2,'004,Afghanistan,221,"Almonds, in shell",5419,Yield,hg/ha,Y2000,17143.0
2,2,'004,Afghanistan,221,"Almonds, in shell",5510,Production,tonnes,Y2000,12000.0
3,2,'004,Afghanistan,711,"Anise, badian, coriander, cumin, caraway, fenn...",5312,Area harvested,ha,Y2000,4000.0
4,2,'004,Afghanistan,711,"Anise, badian, coriander, cumin, caraway, fenn...",5419,Yield,hg/ha,Y2000,6250.0


## Consolidate Data for Sudan and Former Sudan

In [7]:
# Consolidate the data for Sudan and Former Sudan
delete_years_sudan = [f'Y{i}' for i in range(2000, 2012)]
delete_years_former_sudan = [f'Y{i}' for i in range(2012, 2022)]

condition1 = ((df['Area'] == 'Sudan') & (df['variable'].isin(delete_years_sudan)))
condition2 = ((df['Area'] == 'Sudan (former)') & (df['variable'].isin(delete_years_former_sudan)))

combined_condition = condition1 | condition2

rowstodelete = df[combined_condition]

df = df[~combined_condition]

df.replace("Sudan (former)", "Sudan", inplace=True)

## Replace Diacritics

In [8]:
# Replace diacritics
replace_dict = {
    "Côte d'Ivoire": "Cote d'Ivoire",
    "Réunion": "Reunion",
    "Türkiye": "Turkiye"
}

df.replace(replace_dict, inplace=True)

## Final Edits 

In [9]:
# Final edits
df.rename(columns={'variable': 'Year'}, inplace=True)
df['Year'] = df['Year'].str.replace('Y', '')

## Print the DataFrame in CSV

In [10]:
# Export the DataFrame to a new CSV file named "Crop Tableau Data".csv"
df.to_csv('Crop_for_Tableau.csv', index=False)

display(df.head(5))

,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Unit,Year,value
0,2,'004,Afghanistan,221,"Almonds, in shell",5312,Area harvested,ha,2000,7000.0
1,2,'004,Afghanistan,221,"Almonds, in shell",5419,Yield,hg/ha,2000,17143.0
2,2,'004,Afghanistan,221,"Almonds, in shell",5510,Production,tonnes,2000,12000.0
3,2,'004,Afghanistan,711,"Anise, badian, coriander, cumin, caraway, fenn...",5312,Area harvested,ha,2000,4000.0
4,2,'004,Afghanistan,711,"Anise, badian, coriander, cumin, caraway, fenn...",5419,Yield,hg/ha,2000,6250.0
